# Data Part TID
- Events with timestamp and cleared body
- Total- 18,420,783, 
- Unique: UserId - 73584, MsgId - 18420733, 
- Percentage events - 60% UserSignin, 15% TransConfirm, 10% ecobal, 10% nudgecreat, 5% transcreat, 4% risklvl, 4% loyaltylevl
- Date - 2020-02-04 start, end - 2022-06-13

In [1]:
INPATH = '/Users/pranjal/Downloads/data/part-00000-tid-7073521145656742331-3e06173f-1926-41aa-b3b7-847006329acc-4624-1-c000.csv'
column_names = ['UserId', 'IntegrationEvent', 'MessageId', 'CreationDateUtc', 'ClearedBody']

import pandas as pd
import random
p = 0.05
df = pd.read_csv(INPATH, 
                 error_bad_lines = False, 
                 names = column_names, 
                 #nrows = 100000,
                 #skiprows=lambda i: i>0 and random.random() > p
                ) #skiprows = 460953, nrows = 200000)

df = df[1:]
print(df.shape)
df.head()

(18420783, 5)


,UserId,IntegrationEvent,MessageId,CreationDateUtc,ClearedBody
1,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,NudgeCreated,852618c20ec54ef69ad8c7b64f7ad564,2021-10-16T08:09:25.000Z,"{\message\"": \""Con i Gruppi di Spesa \\u00e8 f..."
2,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,NudgeCreated,cb8e47a582b940d3ab93ba9ec8339bc1,2021-10-22T08:14:28.000Z,"{\message\"": \""Segui la rotta dei video Focus ..."
3,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,UserSignedIn,a5e17409-3ac8-4be9-8058-8c0031c14855,2021-12-25T00:17:49.000Z,"{\deviceCode\"": \""a44e3f40-511c-452a-99c8-7be3..."
4,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,UserSignedIn,e1a14dbe-adc8-4eb9-b043-fd7315e9234f,2021-09-30T12:51:36.000Z,"{\deviceCode\"": \""a7ebd2ee-72aa-45fc-9825-246b..."
5,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,UserSignedIn,c58d4b23-da8d-4e62-983a-a7ae1bc7d00f,2021-10-08T08:18:05.000Z,"{\deviceCode\"": \""a44e3f40-511c-452a-99c8-7be3..."


In [3]:
# Cleaning the "Body" json
import json
def clean(x):
    x = x.replace("\\", "")
    x = x.replace("/", "")
    x = x + "}"
    x = x.replace("{", "{\"")
    x = x.replace("\\", "")
    x = x.replace("}", "\"}")
    x = x.replace(": ", ": \"")
    x = x.replace("\"\"", "\"")
    x = x.replace("\"}\"}", "\"}")

    try:
        x = json.loads(x)
        return x
    except:
        print(x)
        return {}

df['ClearedBody'] = df['ClearedBody'].apply(clean)

{"}
{"}
{"}
{"}
{"}
{"}
{"}
{"}


In [4]:
df['CreationDateUtc'] = pd.to_datetime(df.CreationDateUtc)
df['date'] = df['CreationDateUtc'].dt.date
df['yearmonth'] = df['CreationDateUtc'].dt.to_period('M')

/usr/local/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1101: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


# Check 1

In [5]:
# Summary Count 1
df.IntegrationEvent.value_counts()/df.shape[0]

UserSignedIn              0.597986
TransactionConfirmed      0.146223
EcoBalanceDataComputed    0.098023
NudgeCreated              0.089591
TransactionCreated        0.049593
NudgeRead                 0.006892
RiskLevelEvaluated        0.004553
LoyaltyLevelChanged       0.004021
TreeAcquired              0.001200
PhysicalCardRequested     0.001049
SubscriptionActivated     0.000870
Name: IntegrationEvent, dtype: float64

In [6]:
df2 = df.drop_duplicates(subset = ['UserId', 'IntegrationEvent'])

In [7]:
df2.IntegrationEvent.value_counts()/df2.shape[0]

UserSignedIn              0.144387
TransactionConfirmed      0.144387
RiskLevelEvaluated        0.143826
LoyaltyLevelChanged       0.135139
NudgeCreated              0.121897
EcoBalanceDataComputed    0.094824
TransactionCreated        0.072488
TreeAcquired              0.043377
PhysicalCardRequested     0.037567
NudgeRead                 0.036582
SubscriptionActivated     0.025526
Name: IntegrationEvent, dtype: float64

# Separate files and study fields

In [9]:
# Storing the data into event types
eventTypes = list(df.IntegrationEvent.value_counts().index)
OUTPATH = '/Users/pranjal/Downloads/data'
for i in eventTypes:
    df[df.IntegrationEvent == i].to_pickle(OUTPATH + str(i) + '.pkl')

# Panels

In [10]:
df1 = df[df.IntegrationEvent == 'UserSignIn']

In [11]:
df2 = df[df.IntegrationEvent == 'TransactionConfirmed']